#IMPORT

In [1]:
!pip install matplotlib
!pip install gdown
!pip install xgboost

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Admin\AppData\Local\Programs\Python\Python312\Scripts\pip.exe\__main__.py", line 4, in <module>
ModuleNotFoundError: No module named 'pip'
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Admin\AppData\Local\Programs\Python\Python312\Scripts\pip.exe\__main__.py", line 4, in <module>
ModuleNotFoundError: No module named 'pip'
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Admin\AppData\Local\Programs\Python\Python312\Scripts\pip.exe\__main__.py", line 4, in <module>
ModuleNotFoundError: No module named 'pip'


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

from xgboost import XGBRegressor

In [3]:
import pandas as pd
import numpy as np
import os
import gdown
import matplotlib as plt
import re
import glob

#DATA PREPROCESSING

In [10]:
urls = {
    "2020": "https://drive.google.com/uc?id=1ULI0h3EKo5kuUMsT2xFnUD8i202QJjYZ",
    "2021": "https://drive.google.com/uc?id=12M2eDIfMP6wJAnmziGmkl11lwraF68NZ",
    "2022": "https://drive.google.com/uc?id=1cldDmJ7Aknjqw7eC_8T9_kSf2f9ysniG",
    "2023": "https://drive.google.com/uc?id=1nff4fbOEV7O0rIMKV9fAIme26LID0IC_",
    "2024": "https://drive.google.com/uc?id=1-r_Grt20TvrmMEFBaLKjcTxzLBKTnnV7"
}

os.makedirs("./data", exist_ok=True)

for year, url in urls.items():
    output = f"./data/vnexpress_{year}_clearned.csv"
    print(f"Downloading {year}...")
    gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1ULI0h3EKo5kuUMsT2xFnUD8i202QJjYZ
To: d:\Grab\predict\data\vnexpress_2020_clearned.csv
100%|██████████| 301k/301k [00:00<00:00, 1.23MB/s]


Downloading...
From: https://drive.google.com/uc?id=12M2eDIfMP6wJAnmziGmkl11lwraF68NZ
To: d:\Grab\predict\data\vnexpress_2021_clearned.csv
100%|██████████| 344k/344k [00:00<00:00, 1.25MB/s]


Downloading...
From: https://drive.google.com/uc?id=1cldDmJ7Aknjqw7eC_8T9_kSf2f9ysniG
To: d:\Grab\predict\data\vnexpress_2022_clearned.csv
100%|██████████| 375k/375k [00:00<00:00, 1.25MB/s]


Downloading...
From: https://drive.google.com/uc?id=1nff4fbOEV7O0rIMKV9fAIme26LID0IC_
To: d:\Grab\predict\data\vnexpress_2023_clearned.csv
100%|██████████| 448k/448k [00:00<00:00, 1.41MB/s]


Downloading...
From: https://drive.google.com/uc?id=1-r_Grt20TvrmMEFBaLKjcTxzLBKTnnV7
To: d:\Grab\predict\data\vnexpress_2024_clearned.csv
100%|██████████| 622k/622k [00:00<00:00, 1.42MB/s]


In [11]:
all_dfs = []

for file in glob.glob("./data/vnexpress_*_clearned.csv"):
    match = re.search(r"(\d{4})", file)
    if match:
        nam = int(match.group(1))  # Lấy năm tìm được
        print(f"Working with {file} ({nam})...")
        df = pd.read_csv(file)
        df["nam"] = nam
        all_dfs.append(df)
    else:
        print(f"Cannot find {file}")

if all_dfs:
    data = pd.concat(all_dfs, ignore_index=True)
    data = data.drop(['hoc_phi'], axis=1)
    
    print("All the data: ")
    print(data.head())
    
    # Lưu dữ liệu vào file data.csv
    data.to_csv('./data/data.csv', index=False)
    print("Done downloading")
else:
    print("There is nothing to work with")

Working with ./data\vnexpress_2020_clearned.csv (2020)...
Working with ./data\vnexpress_2021_clearned.csv (2021)...
Working with ./data\vnexpress_2022_clearned.csv (2022)...
Working with ./data\vnexpress_2023_clearned.csv (2023)...
Working with ./data\vnexpress_2024_clearned.csv (2024)...
All the data: 
  ma_truong                ten_truong dia_diem ma_nganh  \
0       BKA  Đại học Bách khoa Hà Nội   Hà Nội   CH-E11   
1       BKA  Đại học Bách khoa Hà Nội   Hà Nội    MS-E3   
2       BKA  Đại học Bách khoa Hà Nội   Hà Nội    ET-E4   
3       BKA  Đại học Bách khoa Hà Nội   Hà Nội   EM-E13   
4       BKA  Đại học Bách khoa Hà Nội   Hà Nội    ME-E1   

                                     ten_nganh   diem     to_hop_mon   nam  
0     Chương trình tiên tiến Kỹ thuật Hóa dược  26.50  A00, B00, D07  2020  
1         Chương trình tiên tiến KHKT Vật liệu  23.18  A00, A01, D07  2020  
2  Chương trình tiên tiến Điện tử - Viễn thông  27.15       A00, A01  2020  
3  Chương trình tiên tiến Phân t

In [3]:
csv_path = './data/data.csv'

df = pd.read_csv(csv_path)

nganh_lon_hon_50 = df[(df['diem'] > 50) & (df['ma_truong'] != 'QSB')]
print(f"Số ngành có điểm > 50: {len(nganh_lon_hon_50)}")
print("\nMột số ngành có điểm lớn hơn 50:")
print(nganh_lon_hon_50[['ma_truong', 'ma_nganh', 'ten_nganh', 'diem']].head())

df.loc[(df['ma_truong'] == 'NLS') & (df['ma_nganh'] == '7340101'), 'diem'] = 23.3
df.to_csv(csv_path, index=False)
print("Đã sửa điểm ngành NLS - 7340101 thành 23.3 và lưu lại.")

Số ngành có điểm > 50: 0

Một số ngành có điểm lớn hơn 50:
Empty DataFrame
Columns: [ma_truong, ma_nganh, ten_nganh, diem]
Index: []
Đã sửa điểm ngành NLS - 7340101 thành 23.3 và lưu lại.


In [13]:
#Folder cho các thư mục dự đoán sau này
#Do data đang có các ngành theo thang điểm khác nhau: 30, 40, 90 (qsb)
folders = ['qsb', 'thang_30', 'thang_40']
for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("Create all the folders for data")

Create all the folders for data


In [ ]:
#Only for QSB
def Preprocess_QSB(data_file = './data/data.csv'):
    df = pd.read_csv(data_file)
    df_qsb = df[(df['ma_truong'] == 'QSB') & (df['nam'].isin([2022, 2023, 2024]))]

    # Khúc này xử lý các ngành mới có trong 2024 (idea)
    majors_2022_2023 = set(df_qsb[df_qsb['nam'].isin([2022, 2023])]['ma_nganh'])
    majors_2024 = set(df_qsb[df_qsb['nam'] == 2024]['ma_nganh'])
    new_major_2024 = majors_2024 - majors_2022_2023
    df_qsb['newmajor'] = df_qsb['ma_nganh'].apply(lambda x: 1 if x in new_major_2024 else 0)
    print("New majors that appeared in 2024 (newmajor = 1):")
    print(df_qsb[df_qsb['newmajor'] == 1][['ma_nganh', 'ten_nganh']].drop_duplicates())

    df_qsb.to_csv('qsb/qsb_only.csv', index=False)
    print("Filtered data saved to qsb/qsb_only.csv")

In [15]:
def Preprocess(data_file = './data/data.csv', output='', thang30 = True):
    df = pd.read_csv(data_file)
    if thang30 == True:
        df_other = df[(df['ma_truong'] != 'QSB') & (df['diem'] < 30)]
        print("Data < 30")
    else:
        df_other = df[(df['ma_truong'] != 'QSB') & (df['diem'] >= 30)]
        print("Data >= 30")
    
    majors_2020_2023 = set(df_other[df_other['nam'].isin([2020, 2021, 2022, 2023])]['ma_nganh'])
    majors_2024 = set(df_other[df_other['nam'] == 2024]['ma_nganh'])
    new_major_2024 = majors_2024 - majors_2020_2023
    df_other['newmajor'] = df_other['ma_nganh'].apply(lambda x: 1 if x in new_major_2024 else 0)
    
    print("New majors that appeared in 2024 (newmajor = 1):")
    new_major_2024 = df_other[df_other['newmajor'] == 1]
    print(new_major_2024[['ma_nganh', 'ten_nganh']])
    
    if df_other.empty:
        print("No data found for non-QSB schools in the specified years.")
        return
    
    df_other.to_csv(output, index=False)
    print(f"Filtered data for non-QSB schools saved to {output}")

In [16]:
Preprocess_QSB(data_file = './data/data.csv')
Preprocess(data_file = './data/data.csv', output='./thang_30/thang_30.csv', thang30=True)
Preprocess(data_file = './data/data.csv', output='./thang_40/thang_40.csv', thang30=False)

New majors that appeared in 2024 (newmajor = 1):
      ma_nganh              ten_nganh
13203      147  Địa Kỹ thuật xây dựng
13205      146       Khoa học dữ liệu
13211      148       Kinh tế xây dựng
Filtered data saved to qsb/qsb_only.csv
Data < 30
New majors that appeared in 2024 (newmajor = 1):
            ma_nganh                                          ten_nganh
12732            MS5                                        Kỹ thuật in
12736            EM1                                 Quản lý năng lượng
12740            ED3                                   Quản lý giáo dục
12750          NTH10  Định hướng nghề nghiệp Khoa học máy tính và dữ...
12754          NTH06                                Ngôn ngữ Trung Quốc
...              ...                                                ...
17555  7860226|21A00  Chỉ huy Kỹ thuật Phòng không - Không quân (Nam...
17559        7310109                                         Kinh tế số
17562        7310109                                

C:\Users\Admin\AppData\Local\Temp\ipykernel_12728\3062718917.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qsb['newmajor'] = df_qsb['ma_nganh'].apply(lambda x: 1 if x in new_major_2024 else 0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_12728\3154273150.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_other['newmajor'] = df_other['ma_nganh'].apply(lambda x: 1 if x in new_major_2024 else 0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_12728\3154273150.py:13: SettingWithCopyWarning: 
A value i

In [18]:
def extract_subject_combinations(file_path, save_txt=True):
    df = pd.read_csv(file_path)

    def extract_combinations(to_hop_mon):
        if isinstance(to_hop_mon, str):
            return [combo.strip() for combo in to_hop_mon.split(',')]
        return []

    all_combinations = set()
    for to_hop_mon in df['to_hop_mon']:
        combinations = extract_combinations(to_hop_mon)
        all_combinations.update(combinations)

    unique_combinations = sorted(list(all_combinations))
    print("List of unique subject combinations:")
    for combo in unique_combinations:
        print(combo)

    if save_txt:
        txt_path = os.path.splitext(file_path)[0] + '.txt'
        os.makedirs(os.path.dirname(txt_path), exist_ok=True)
        with open(txt_path, 'w') as f:
            for combo in unique_combinations:
                f.write(combo + '\n')
        print(f"\nList saved to '{txt_path}'")

    return unique_combinations

In [19]:
extract_subject_combinations('qsb/qsb_only.csv')
extract_subject_combinations('thang_30/thang_30.csv')
extract_subject_combinations('thang_40/thang_40.csv')

List of unique subject combinations:
A00
A01
B00
B08
D01
D07

List saved to 'qsb/qsb_only.txt'
List of unique subject combinations:
A00
A01
A02
A03
A04
A05
A06
A07
A08
A09
A10
A11
A12
A14
A15
A16
A17
B00
B01
B02
B03
B04
B05
B08
C00
C01
C02
C03
C04
C05
C08
C09
C10
C13
C14
C15
C19
C20
D01
D02
D03
D04
D05
D06
D07
D08
D09
D10
D11
D12
D13
D14
D15
D19
D22
D23
D24
D25
D26
D28
D29
D33
D42
D43
D44
D45
D63
D64
D66
D71
D72
D78
D80
D81
D82
D83
D84
D90
D91
D96
D97
H00
H01
H02
H03
H04
H05
H06
H07
H08
K01
M00
M01
M02
M03
M04
M05
M06
M07
M08
M09
M10
M11
M13
M14
N00
N01
N03
N05
R01
R02
R03
R04
S00
T00
T01
T02
T03
T04
T05
T06
T07
V00
V01
V02
V03
V04
V05
V06
V07
V08
V09
V10

List saved to 'thang_30/thang_30.txt'
List of unique subject combinations:
A00
A01
A02
A06
A16
B00
B03
B08
C00
C01
C03
C04
C14
C15
C19
D01
D02
D03
D04
D05
D06
D07
D08
D09
D10
D11
D12
D14
D15
D55
D66
D72
D78
D84
D90
D96
H00
H01
H02
H03
M00
M01
M05
M09
N00
T00
T01
T02
T05
V00
V01
V02

List saved to 'thang_40/thang_40.txt'


['A00',
 'A01',
 'A02',
 'A06',
 'A16',
 'B00',
 'B03',
 'B08',
 'C00',
 'C01',
 'C03',
 'C04',
 'C14',
 'C15',
 'C19',
 'D01',
 'D02',
 'D03',
 'D04',
 'D05',
 'D06',
 'D07',
 'D08',
 'D09',
 'D10',
 'D11',
 'D12',
 'D14',
 'D15',
 'D55',
 'D66',
 'D72',
 'D78',
 'D84',
 'D90',
 'D96',
 'H00',
 'H01',
 'H02',
 'H03',
 'M00',
 'M01',
 'M05',
 'M09',
 'N00',
 'T00',
 'T01',
 'T02',
 'T05',
 'V00',
 'V01',
 'V02']

#TRAIN VÀ PREDICT

In [4]:
def predict_admission_scores(csv_file, combo_file, output_file, train_years):
    # ==== B1: Đọc dữ liệu ====
    with open(combo_file, 'r') as f:
        common_combinations = [line.strip() for line in f if line.strip()]
    print(" Danh sách tổ hợp môn:", common_combinations)

    df = pd.read_csv(csv_file)

    # ==== B2: Tiền xử lý dữ liệu ====
    le_nganh = LabelEncoder()
    df['ma_nganh_encoded'] = le_nganh.fit_transform(df['ma_nganh'])

    for combo in common_combinations:
        df[f'has_{combo}'] = df['to_hop_mon'].apply(lambda x: 1 if combo in x else 0)

    df = df.sort_values(by=['ma_nganh', 'to_hop_mon', 'nam']).reset_index(drop=True)
    df['nam_truoc'] = df.groupby(['ma_nganh', 'to_hop_mon'])['nam'].shift(1)
    df['diem_truoc'] = df.groupby(['ma_nganh', 'to_hop_mon'])['diem'].shift(1)
    df['nam_truoc'].fillna(0, inplace=True) 
    df['diem_truoc'].fillna(0, inplace=True) 
    df['diem_trend'] = np.where(df['nam'] - df['nam_truoc'] == 1,
                                df['diem'] - df['diem_truoc'], 0)   

    print("\n>>> Dữ liệu sau khi shift và tính trend:")
    print(df[['ma_nganh', 'to_hop_mon', 'nam', 'diem', 'nam_truoc', 'diem_truoc', 'diem_trend']].head(30))

    # ==== B3: Huấn luyện model XGBoost ====
    train_df = df[df['nam'].isin(train_years)]

    features = ['nam', 'ma_nganh_encoded', 'diem_trend', 'newmajor'] + [f'has_{combo}' for combo in common_combinations]
    X_train = train_df[features]
    y_train = train_df['diem']

    xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [3, 6, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.7, 1.0],
        'colsample_bytree': [0.7, 1.0]
    }

    grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid,
                               cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)
    best_xgb_model = grid_search.best_estimator_
    print("\n Tham số tốt nhất:", grid_search.best_params_)

    # ==== B4: Chuẩn bị test_df_2025 từ dữ liệu 2024 ====
    last_year = max(train_years)
    test_year = last_year + 1

    test_df = df[df['nam'] == last_year].copy()
    test_df['nam'] = test_year

    trend_mean = df.groupby('ma_nganh')['diem_trend'].mean().reset_index()
    trend_mean = trend_mean.rename(columns={'diem_trend': 'diem_trend_mean'})
    test_df = pd.merge(test_df, trend_mean, on='ma_nganh', how='left')

    test_df['diem_trend'] = np.where(
        test_df['newmajor'] == 0,
        test_df['diem_trend_mean'].fillna(0),
        0
    )

    test_df = test_df.drop(columns=['diem_trend_mean'])

    # ==== B5: Dự đoán điểm chuẩn năm test_year ====
    X_test = test_df[features]
    test_df['predicted_diem'] = np.round(best_xgb_model.predict(X_test), 2)
    
    # ==== B6: Xuất file kết quả ====
    output_df = test_df[['ma_truong', 'ten_truong', 'dia_diem', 'ma_nganh', 'ten_nganh',
                         'to_hop_mon', 'predicted_diem', 'newmajor']]
    output_df['nam'] = test_year
    output_df.to_csv(output_file, index=False)
    print(f" Đã lưu dự đoán năm {test_year} vào '{output_file}'.")

    # ==== B7: Kiểm tra nếu có dữ liệu thật cho test_year và tính metric ====
    if test_year in df['nam'].values:
        # Lấy điểm thật theo năm và đúng tổ hợp
        true_values_df = df[df['nam'] == test_year][['ma_nganh', 'to_hop_mon', 'diem']]

        test_df = pd.merge(
            test_df,
            true_values_df,
            on=['ma_nganh', 'to_hop_mon'],
            how='left',
            suffixes=('', '_true')
        )

        # Lọc ra những ngành + tổ hợp có điểm thật
        valid_test_df = test_df.dropna(subset=['diem_true'])

        if len(valid_test_df) > 0:
            mae = mean_absolute_error(valid_test_df['diem_true'], valid_test_df['predicted_diem'])
            mse = mean_squared_error(valid_test_df['diem_true'], valid_test_df['predicted_diem'])
            r2 = r2_score(valid_test_df['diem_true'], valid_test_df['predicted_diem'])

            print(f" MAE: {mae}")
            print(f" MSE: {mse}")
            print(f" R^2: {r2}")
        else:
            print(" Không có đủ ngành hợp lệ để tính metric (ngành mới không có dữ liệu thật).")
    else:
        print(f" Không có dữ liệu thật cho năm {test_year}, không tính được các metric.")


    # ==== B7: In danh sách ngành mới ====
    print(f"\n Các ngành mới (newmajor == 1) trong năm {test_year} và điểm dự đoán:")
    print(output_df[output_df['newmajor'] == 1][['ma_nganh', 'ten_nganh', 'predicted_diem']].to_string(index=False))


#QSB

In [5]:
predict_admission_scores('qsb/qsb_only.csv', 'qsb/qsb_only.txt', 'qsb/2024_qsb.csv', train_years=[2022, 2023])

 Danh sách tổ hợp môn: ['A00', 'A01', 'B00', 'B08', 'D01', 'D07']

>>> Dữ liệu sau khi shift và tính trend:
    ma_nganh          to_hop_mon   nam   diem  nam_truoc  diem_truoc  \
0        106            A00, A01  2022  75.99        0.0        0.00   
1        106            A00, A01  2023  79.84     2022.0       75.99   
2        106            A00, A01  2024  84.16     2023.0       79.84   
3        107            A00, A01  2022  66.86        0.0        0.00   
4        107            A00, A01  2024  82.87     2022.0       66.86   
5        108            A00, A01  2022  60.00        0.0        0.00   
6        108            A00, A01  2023  66.59     2022.0       60.00   
7        108            A00, A01  2024  80.03     2023.0       66.59   
8        109            A00, A01  2022  60.29        0.0        0.00   
9        109            A00, A01  2023  58.49     2022.0       60.29   
10       109            A00, A01  2024  73.89     2023.0       58.49   
11       110            A00,

C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['nam_truoc'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w


 Tham số tốt nhất: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
 Đã lưu dự đoán năm 2024 vào 'qsb/2024_qsb.csv'.
 MAE: 7.642500171661377
 MSE: 82.06188683944902
 R^2: -0.2504557049215328

 Các ngành mới (newmajor == 1) trong năm 2024 và điểm dự đoán:
Empty DataFrame
Columns: [ma_nganh, ten_nganh, predicted_diem]
Index: []


C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['nam'] = test_year


In [6]:
predict_admission_scores('qsb/qsb_only.csv', 'qsb/qsb_only.txt', 'qsb/2025_qsb.csv', train_years=[2022, 2023, 2024])

 Danh sách tổ hợp môn: ['A00', 'A01', 'B00', 'B08', 'D01', 'D07']

>>> Dữ liệu sau khi shift và tính trend:
    ma_nganh          to_hop_mon   nam   diem  nam_truoc  diem_truoc  \
0        106            A00, A01  2022  75.99        0.0        0.00   
1        106            A00, A01  2023  79.84     2022.0       75.99   
2        106            A00, A01  2024  84.16     2023.0       79.84   
3        107            A00, A01  2022  66.86        0.0        0.00   
4        107            A00, A01  2024  82.87     2022.0       66.86   
5        108            A00, A01  2022  60.00        0.0        0.00   
6        108            A00, A01  2023  66.59     2022.0       60.00   
7        108            A00, A01  2024  80.03     2023.0       66.59   
8        109            A00, A01  2022  60.29        0.0        0.00   
9        109            A00, A01  2023  58.49     2022.0       60.29   
10       109            A00, A01  2024  73.89     2023.0       58.49   
11       110            A00,

C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['nam_truoc'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w


 Tham số tốt nhất: {'colsample_bytree': 0.7, 'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.7}
 Đã lưu dự đoán năm 2025 vào 'qsb/2025_qsb.csv'.
 Không có dữ liệu thật cho năm 2025, không tính được các metric.

 Các ngành mới (newmajor == 1) trong năm 2025 và điểm dự đoán:
 ma_nganh             ten_nganh  predicted_diem
      146      Khoa học dữ liệu       77.800003
      147 Địa Kỹ thuật xây dựng       55.700001
      148      Kinh tế xây dựng       61.070000


C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['nam'] = test_year


#THANG_40

In [7]:
predict_admission_scores('thang_40/thang_40.csv', 'thang_40/thang_40.txt', 'thang_40/2024_thang_40.csv', train_years=[2020, 2021, 2022, 2023])


 Danh sách tổ hợp môn: ['A00', 'A01', 'A02', 'A06', 'A16', 'B00', 'B03', 'B08', 'C00', 'C01', 'C03', 'C04', 'C14', 'C15', 'C19', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D14', 'D15', 'D55', 'D66', 'D72', 'D78', 'D84', 'D90', 'D96', 'H00', 'H01', 'H02', 'H03', 'M00', 'M01', 'M05', 'M09', 'N00', 'T00', 'T01', 'T02', 'T05', 'V00', 'V01', 'V02']

>>> Dữ liệu sau khi shift và tính trend:
      ma_nganh to_hop_mon   nam   diem  nam_truoc  diem_truoc  diem_trend
0  FA7340120   A01, D01  2021  33.50        0.0        0.00        0.00
1  FA7340120   A01, D01  2023  31.50     2021.0       33.50        0.00
2          602        D01  2020  31.00        0.0        0.00        0.00
3          602        D01  2022  35.35     2020.0       31.00        0.00
4          602        D01  2023  34.47     2022.0       35.35       -0.88
5          602        D01  2024  35.48     2023.0       34.47        1.01
6          602        D72  2020  31.00        0.0       

C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['nam_truoc'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w


 Tham số tốt nhất: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'subsample': 0.7}
 Đã lưu dự đoán năm 2024 vào 'thang_40/2024_thang_40.csv'.
 MAE: 0.95776979679684
 MSE: 1.4779316038514982
 R^2: 0.5873923851626186

 Các ngành mới (newmajor == 1) trong năm 2024 và điểm dự đoán:
Empty DataFrame
Columns: [ma_nganh, ten_nganh, predicted_diem]
Index: []


C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['nam'] = test_year


In [8]:
predict_admission_scores('thang_40/thang_40.csv', 'thang_40/thang_40.txt', 'thang_40/2025_thang_40.csv', train_years=[2020, 2021, 2022, 2023, 2024])


 Danh sách tổ hợp môn: ['A00', 'A01', 'A02', 'A06', 'A16', 'B00', 'B03', 'B08', 'C00', 'C01', 'C03', 'C04', 'C14', 'C15', 'C19', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D14', 'D15', 'D55', 'D66', 'D72', 'D78', 'D84', 'D90', 'D96', 'H00', 'H01', 'H02', 'H03', 'M00', 'M01', 'M05', 'M09', 'N00', 'T00', 'T01', 'T02', 'T05', 'V00', 'V01', 'V02']

>>> Dữ liệu sau khi shift và tính trend:
      ma_nganh to_hop_mon   nam   diem  nam_truoc  diem_truoc  diem_trend
0  FA7340120   A01, D01  2021  33.50        0.0        0.00        0.00
1  FA7340120   A01, D01  2023  31.50     2021.0       33.50        0.00
2          602        D01  2020  31.00        0.0        0.00        0.00
3          602        D01  2022  35.35     2020.0       31.00        0.00
4          602        D01  2023  34.47     2022.0       35.35       -0.88
5          602        D01  2024  35.48     2023.0       34.47        1.01
6          602        D72  2020  31.00        0.0       

C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['nam_truoc'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w


 Tham số tốt nhất: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.7}
 Đã lưu dự đoán năm 2025 vào 'thang_40/2025_thang_40.csv'.
 Không có dữ liệu thật cho năm 2025, không tính được các metric.

 Các ngành mới (newmajor == 1) trong năm 2025 và điểm dự đoán:
   ma_nganh                                                 ten_nganh  predicted_diem
    7140208                             Giáo dục Quốc phòng - An ninh       35.750000
 7220201 TT                                       Ngôn ngữ Anh - CTTT       32.860001
7220201_WE2                                       CTLK - Ngôn ngữ Anh       30.629999
7220201_WE3                                       CTLK - Ngôn ngữ Anh       30.629999
7220201_WE4                                       CTLK - Ngôn ngữ Anh       30.629999
 7220204 TT                                Ngôn ngữ Trung Quốc - CTTT       33.639999
 7220208 TT                                    Ngôn ngữ Italia - CTTT       32.200001
 

C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['nam'] = test_year


#THANG_30

In [9]:
predict_admission_scores('thang_30/thang_30.csv', 'thang_30/thang_30.txt', 'thang_30/2024_thang_30.csv', train_years=[2020, 2021, 2022, 2023])


 Danh sách tổ hợp môn: ['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A14', 'A15', 'A16', 'A17', 'B00', 'B01', 'B02', 'B03', 'B04', 'B05', 'B08', 'C00', 'C01', 'C02', 'C03', 'C04', 'C05', 'C08', 'C09', 'C10', 'C13', 'C14', 'C15', 'C19', 'C20', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D19', 'D22', 'D23', 'D24', 'D25', 'D26', 'D28', 'D29', 'D33', 'D42', 'D43', 'D44', 'D45', 'D63', 'D64', 'D66', 'D71', 'D72', 'D78', 'D80', 'D81', 'D82', 'D83', 'D84', 'D90', 'D91', 'D96', 'D97', 'H00', 'H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'K01', 'M00', 'M01', 'M02', 'M03', 'M04', 'M05', 'M06', 'M07', 'M08', 'M09', 'M10', 'M11', 'M13', 'M14', 'N00', 'N01', 'N03', 'N05', 'R01', 'R02', 'R03', 'R04', 'S00', 'T00', 'T01', 'T02', 'T03', 'T04', 'T05', 'T06', 'T07', 'V00', 'V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09', 'V10']


C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'has_{combo}'] = df['to_hop_mon'].apply(lambda x: 1 if combo in x else 0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'has_{combo}'] = df['to_hop_mon'].apply(lambda x: 1 if combo in x else 0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of


>>> Dữ liệu sau khi shift và tính trend:
       ma_nganh          to_hop_mon   nam   diem  nam_truoc  diem_truoc  \
0   FA7340120            A01, D01  2022  27.00        0.0        0.00   
1   FA7340120            A01, D01  2024  28.00     2022.0       27.00   
2      51140201  C00, C10, C20, D01  2024  21.00        0.0        0.00   
3      51140201  C00, C14, C19, C20  2023  17.00        0.0        0.00   
4      51140201  C00, C14, C19, C20  2024  24.68     2023.0       17.00   
5      51140201                 M00  2024  25.80        0.0        0.00   
6      51140201                 M01  2020  16.50        0.0        0.00   
7      51140201                 M01  2021  17.34     2020.0       16.50   
8      51140201                 M01  2023  20.35     2021.0       17.34   
9      51140201                 M01  2024  24.00     2023.0       20.35   
10     51140201                 M05  2024  20.50        0.0        0.00   
11  51140201_LA            M02, M03  2024  18.70        0.0   

C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['nam'] = test_year


In [10]:
predict_admission_scores('thang_30/thang_30.csv', 'thang_30/thang_30.txt', 'thang_30/2025_thang_30.csv', train_years=[2020, 2021, 2022, 2023, 2024])


 Danh sách tổ hợp môn: ['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A14', 'A15', 'A16', 'A17', 'B00', 'B01', 'B02', 'B03', 'B04', 'B05', 'B08', 'C00', 'C01', 'C02', 'C03', 'C04', 'C05', 'C08', 'C09', 'C10', 'C13', 'C14', 'C15', 'C19', 'C20', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D19', 'D22', 'D23', 'D24', 'D25', 'D26', 'D28', 'D29', 'D33', 'D42', 'D43', 'D44', 'D45', 'D63', 'D64', 'D66', 'D71', 'D72', 'D78', 'D80', 'D81', 'D82', 'D83', 'D84', 'D90', 'D91', 'D96', 'D97', 'H00', 'H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'K01', 'M00', 'M01', 'M02', 'M03', 'M04', 'M05', 'M06', 'M07', 'M08', 'M09', 'M10', 'M11', 'M13', 'M14', 'N00', 'N01', 'N03', 'N05', 'R01', 'R02', 'R03', 'R04', 'S00', 'T00', 'T01', 'T02', 'T03', 'T04', 'T05', 'T06', 'T07', 'V00', 'V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09', 'V10']


C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'has_{combo}'] = df['to_hop_mon'].apply(lambda x: 1 if combo in x else 0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'has_{combo}'] = df['to_hop_mon'].apply(lambda x: 1 if combo in x else 0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of


>>> Dữ liệu sau khi shift và tính trend:
       ma_nganh          to_hop_mon   nam   diem  nam_truoc  diem_truoc  \
0   FA7340120            A01, D01  2022  27.00        0.0        0.00   
1   FA7340120            A01, D01  2024  28.00     2022.0       27.00   
2      51140201  C00, C10, C20, D01  2024  21.00        0.0        0.00   
3      51140201  C00, C14, C19, C20  2023  17.00        0.0        0.00   
4      51140201  C00, C14, C19, C20  2024  24.68     2023.0       17.00   
5      51140201                 M00  2024  25.80        0.0        0.00   
6      51140201                 M01  2020  16.50        0.0        0.00   
7      51140201                 M01  2021  17.34     2020.0       16.50   
8      51140201                 M01  2023  20.35     2021.0       17.34   
9      51140201                 M01  2024  24.00     2023.0       20.35   
10     51140201                 M05  2024  20.50        0.0        0.00   
11  51140201_LA            M02, M03  2024  18.70        0.0   

C:\Users\Admin\AppData\Local\Temp\ipykernel_14312\1930885332.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['nam'] = test_year


#OUTPUT

In [25]:
file_paths = [
    './qsb/2025_qsb.csv',
    './thang_30/2025_thang_30.csv',
    './thang_40/2025_thang_40.csv'
]

dfs = []
for path in file_paths:
    df = pd.read_csv(path)
    dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)
merged_df = merged_df.rename(columns={'predicted_diem': 'diem'})
output_df = merged_df[['ma_truong', 'ten_truong', 'dia_diem', 'ma_nganh', 'ten_nganh', 'diem', 'to_hop_mon']]
output_df.to_csv('./data/predict_2025.csv', index=False)
print(" Đã lưu file kết quả tại './data/predict_2025.csv'.")


 Đã lưu file kết quả tại './data/predict_2025.csv'.
